# CoreNLP Stanza

Esta herramienta de la Universidad de Stanford nos permite tokenizar el texto, etiquetar las partes del discurso y buscar entidades nombradas. Su documentación puede encontrarse en https://stanfordnlp.github.io/stanza/index.html .

In [1]:
import stanza
from pymongo import MongoClient, UpdateOne
from pprint import pprint
import re

## Descarga y configuración de Stanza

In [3]:
stanza.download('es', processors='tokenize,mwt,pos,ner')

2020-05-22 16:00:00 INFO: Downloading these customized packages for language: es (Spanish)...
| Processor       | Package  |
------------------------------
| tokenize        | ancora   |
| mwt             | ancora   |
| pos             | ancora   |
| ner             | conll02  |
| pretrain        | ancora   |
| backward_charlm | newswiki |
| forward_charlm  | newswiki |

2020-05-22 18:13:31 INFO: Finished downloading models and saved to C:\Users\Usuario.000\stanza_resources.


In [3]:
nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,ner', use_gpu=True)

2020-05-23 11:36:14 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| ner       | conll02 |

2020-05-23 11:36:14 INFO: Use device: cpu
2020-05-23 11:36:14 INFO: Loading: tokenize
2020-05-23 11:36:14 INFO: Loading: mwt
2020-05-23 11:36:14 INFO: Loading: pos
2020-05-23 11:36:16 INFO: Loading: ner
2020-05-23 11:36:20 INFO: Done loading processors!


## Conexión a Base de Datos

In [2]:
uri = open('./../../mongo_uri.txt', 'r', encoding='utf-8').read()
client = MongoClient(uri)
print(client)
db = client.boletines_db

MongoClient(host=['clustershido-shard-00-02-uxkjr.gcp.mongodb.net:27017', 'clustershido-shard-00-01-uxkjr.gcp.mongodb.net:27017', 'clustershido-shard-00-00-uxkjr.gcp.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='ClusterShido-shard-0', ssl=True)


In [32]:
cursor = db.boletines.find(
    {}, 
    {'boletin':1, 'encabezado':1, 'sumarios':1}
).sort('fecha').limit(2)

In [16]:
# Agrego punto final a oraciones que no tienen
patron_p = re.compile(r'([^\.])(\n|$)')
remplazo_p = r'\1.\n'

patron_com = re.compile(r'[\u201c\u201d]')
remplazo_com = r'"'

In [33]:
def procesar_sent(txt, nlp):
    doc = nlp(txt)
    return [
        { 
            'tokens' : [
                token.to_dict() for token in sent.tokens
            ],
            'entities' : [
                ent.to_dict() for ent in sent.entities
            ],
        }
        for sent in doc.sentences
    ]

In [34]:
%%time
updates = []
i=0

for boletin in cursor:
    filtro = {'boletin_id': boletin['_id']}
    update = {'$set' : {}}
    
    # Encabezado
    txt = patron_p.sub(remplazo_p, boletin['encabezado'])
    txt = patron_com.sub(remplazo_com, txt)
    # Procesar oración
    update['$set']['encabezado'] = procesar_sent(txt, nlp)
    
    # Sumarios
    try:
        txt = patron_p.sub(remplazo_p, '\n'.join(boletin['sumarios']))
        txt = patron_com.sub(remplazo_com, txt)
        sentences_sum = procesar_sent(txt, nlp)
        update['$set']['sumarios'] = sentences_sum
    except KeyError:
        sentences_sum = []
    
    # Contenido principal
    txt = patron_com.sub(remplazo_com, boletin['boletin'])
    # Procesar oración
    update['$set']['boletin'] = procesar_sent(txt, nlp)
    
    if i%1==0:
    #Ejemplo, obtener todos los lemas
        for sentences in [
            update['$set']['encabezado'], 
            sentences_sum, 
            update['$set']['boletin']
        ]:
            for i_sent, sentence in enumerate(sentences):
                print(f'==== Oración {i_sent+1} ====')
                
                tokens = [token[0]['text'] for token in sentence['tokens']]
                # tokens = sentence['tokens']
                print('--Tokens:')
                # print(tokens)
                print(' '.join(tokens))
                
                nes = [ent['text'] for ent in sentence['entities']]
                # nes = sentence['entities']
                if len(nes) > 0:
                    print('--Named Entities:')
                    # print(nes)
                    print('\n'.join(nes))
            print('--------------------------')
        print('==========================================\n')
    
    updates.append(UpdateOne(filtro, update, upsert = True)) 
    i+=1
cursor.rewind()

==== Oración 1 ====
--Tokens:
José Antonio Meade presentó la Coordinación de Atención Ciudadana y Buzón del Candidato .
--Named Entities:
José Antonio Meade
Coordinación de Atención Ciudadana
Buzón del Candidato
--------------------------
==== Oración 1 ====
--Tokens:
El abanderado presidencial refrendó su compromiso de encabezar un gobierno a la medida de cada persona .
--------------------------
==== Oración 1 ====
--Tokens:
El candidato de la coalición Todos por México a la Presidencia de la República , José Antonio Meade , presentó la Coordinación de Atención Ciudadana y el Buzón del Candidato .
--Named Entities:
Todos por México
Presidencia de la República
José Antonio Meade
Coordinación de Atención Ciudadana
Buzón del Candidato
==== Oración 2 ====
--Tokens:
A través de estos mecanismos , Meade y su esposa , Juana Cuevas , recabarán las propuestas , opiniones , inquietudes y necesidades de la población .
--Named Entities:
Meade
Juana Cuevas
==== Oración 3 ====
--Tokens:
El equipo 

In [22]:
cursor.rewind()

In [11]:
%%time
test = """Andrés Manuel López Obrador observó que en el segundo debate le fue peor a Anaya que a Meade, porque Anaya quedó al descubierto como un mentecato, un mentiroso y “nosotros para arriba como la espuma del mar”.
En entrevista y a la pregunta de los reporteros que ayer se apagó el amor y paz, porque respondió a los señalamientos de Anaya y Meade, el candidato por la coalición “Juntos haremos historia” manifestó que se pasó de la raya el candidato del PAN.
“Me dijeron de que era amor y paz, pero que yo no de dejará, mis asesores del pueblo me aconsejaron, me dijeron sí se portan altaneros y groseros, ponlos en su lugar, dales un estate quieto”, dijo.
Sobre el reportaje que fue publicado en la revista Proceso sobre que no concuerdan los ingresos de Anaya con lo que declara, López Obrador respondió que presentó un libro que circula sobre las mentiras de Anaya y será un capítulo por día, porque miente un día sí y el otro también.
A la pregunta de los reporteros qué si ya es de dos la contienda después del debate, ya es con Anaya, pareciera que a Meade lo ignoró, el candidato presidencial respondió que no, solo que el que se portó muy majadero fue Anaya, estaba llegando, y se le para enfrente, a sacarle que no era cierto lo que decía que había habido inversión extranjera cuando fue Jefe de Gobierno.
Sobre sí fue agresivo Anaya, López Obrador indicó que fue muy hipócrita, están acostumbrados a mentir, apantallar, muy mal y lamentó que actúe de esa forma.
López Obrador aseguró que salió sano y salvo, así como preservó su cartera luego del segundo debate presidencial que se realizó en Tijuana, Baja California y dio a conocer que la última etapa de su campaña se basar en “amor y paz”.
A la pregunta de los reporteros sobre que la cantante Belinda dijo que ganó el debate, López Obrador le agradeció mucho a Belinda por su apoyo, “le gradezco mucho a Belinda, es una extraordinaria artista, muy buena cantante, una mujer un inteligente y le agradezco mucho su apoyo, su solidaridad” y agregó que Belinda tienen muchos seguidores inteligentes, “ese apoyo sí se ve.
En otro tema, el candidato a la Presidencia de México invitó a Alicia Bárcena para que cuando concluya su función en la CEPAL sea la embajadora en la Organización de las Naciones Unidas y dio a conocer que conoció a Bárcena en la República de Chile.
Detalló que Bárcena ya estuvo en la ONU, además es la secretaria en la CEPAL, es un mujer muy inteligente, además es una persona con convicciones.
Explicó que recurrirá cuando sea necesario a la ONU sobre todo para defender derechos humanos a migrantes”, desaparecidos, violación de derechos humanos y en el tema de transparencia para dar seguimiento de los contratos y que no haya corrupción, especificó.
En su participación en La Barca, Jalisco López Obrador convocó a los dirigentes de la coalición “Juntos haremos historia” integrado por los partidos: MORENA, PT y PES a llevar a una campaña para que orienten y enseñen a los ciudadanos de cómo votar el próximo domingo primero de julio.
La campaña, describió, servirá para que no se anulen los votos, el voto sea efectivo, no vaya haber problemas a la hora de votar.
Pidió a los ciudadanos que se apunten para ser representantes de casillas, que vayan temprano a votar y que por la tarde-noche se tenga las actas en las manos para que no haya fraude y que orienten a la gente para que no vendan sus votos a cambio de despensas.
“Eso que dan es pan para hoy, pero hambre para mañana, el pueblo lo que requiere es justicia”, mencionó al llamar a los habitantes de La Barca a votar por los candidatos de la coalición “Juntos haremos historia”.
López Obrador se pronunció por la unidad del pueblo de México en el partido de la transformación del país, después de que se triunfe, cada quien se puede regresar a su partido a seguir peleando y dejó en claro que es respetuoso de la libertad de creencias.
Durante su participación en La Barca, Jalisco, el candidato a la Presidencia de México manifestó que ya va a dar el último jalón, está 25 puntos arriba de ventaja, está en primero lugar en Jalisco, en Michoacán, en todo el país, ya la gente ya decidió, “este arroz ya se coció”.
“De todas maneras, no nos vamos a confiar, porque es una mafia de poder que no quiere dejar de robar, no tienen llenadera, son barriles sin fondo, no quiere perder privilegios, sobre todo no quieren perder el privilegio de mandar, porque ellos se sienten los dueños de México”, planteó.
“Ya estamos a 39 días para iniciar la transformación de México, después de muchos años de lucha, por fin vamos a iniciar una transformación de la vida pública de México”, indicó al decir que millones de mexicanos llevarán a cabo la cuarta transformación de la vida pública, sin violencia, de manera pacífica
Expuso que no se engañará a la gente, no habrá bipartidismo, ni alternativa donde nada más entra otro partido, pero sigue la misma corrupción y lo que se necesita es una transformación para que las cosas realmente mejoren en el país.
“Vamos a empezar a desterrar la corrupción del país, se hará el acuerdo que la honestidad se utilice como forma de vida y de gobierno”, explicó.
Se bajará el sueldo de los de arriba, para aumentar los sueldos de los de abajo, ganarán más los maestros, las enfermeras, los soldados, el sueldo de los trabajadores al servicio del Estado, esto se llama justicia laboral.
Si bajar el sueldo de los de arriba para aumentar el sueldo de los de abajo es ser populista, que me apunten en la lista
Adelantó que se cortará el copete de los privilegios, no usará el avión presidencial de Enrique Peña de 7 mil 500 millones de pesos, venderá la flotilla de aeronaves del gobierno federal y estará prohibido que los altos funcionarios utilicen los aviones y helicópteros para trasladarse.
Dijo que no vivirá en Los Pinos, porque esa casa está embrujada, sale “el chupacabras”, no utilizará los servicios del Estado Mayor Presidencial, ya no habrá lujos, se terminará con las pensiones millonarias de los ex presidentes, porque son 5 millones de pesos de pensión a cada ex presidente.
Añadió que ya no se darán 5 mil millones de pesos para la atención médica de los altos funcionarios públicos, irán al ISSSTE, al IMSS o al Seguro Popular.
“Va a cambiar esto y de esta manera vamos a tener presupuesto, nos va a rendir presupuesto”, porque no va a haber corrupción, ni habrá derroche, no aumentará impuestos, ni se endeudará al país, ni habrá gasolinazos, expuso.
El presupuesto se destinará a lo indispensable como es el rescate del campo del abandono, expresó al comprometerse que se garantizará los precios de garantía a los productores del campo, porque México será autosuficiente, cambiará la política agropecuaria, en el país se producirá todo lo que se consume.
Asimismo, López Obrador dio a conocer Víctor Villalobos será el próximo secretario de Agricultura y habrá de nuevo subsidio al campo, solo se tiene que pensar bien para que llegue a todos los productores, así como se apoyará a los ganadores.
Expresó que se la dará estudio y trabajo a 2 millones 600 mil jóvenes, con una inversión de 110 mil millones de pesos, con esto se evitará que la juventud tome el camino de las conductas antisociales y lo resumió en una frase: “becarios sí, sicarios no, nunca más se va a dejar si opciones a los jóvenes”.
Se comprometió a revisar todo lo que han hecho mal todos los gobiernos neoliberales y entrando, entrando, se cancelará la mal llamada reforma educativa, porque no se puede imponer la reforma educativa sin los maestros.
“Vamos a llevar a cabo una auténtica reforma educativa, nos vamos a poner de acuerdo con los maestros, con los padres de familia, con especialistas pedagogos, vamos a elaborar un plan educativo para mejorar de verdad la educación en México, sin afectar los derechos laborales del magisterio”, expuso.
Dijo que ya no se privatizará nada, ni el agua, ni el ISSSTE, ni el Seguro, se mejorará los servicios, aumentará al doble la pensión para los adultos mayores, es decir, serán mil 500 pesos mensuales y será universal.
Señaló que Meade comentó que aumentará al doble el apoyo de los adultos mayores, pero por qué no lo hizo cuando fue secretario de Hacienda e incluso estuvo hasta la Secretaría de Desarrollo Social, ahora porque está necesitando los votos.
Prosiguió que está como Anaya, quien dijo que bajará el precio de las gasolinas, pero si ellos aprobaron el incremento de los hidrocarburos, “ahora están apurados, pues que le sigan, porque ya se les a cabo el tiempo, estamos como 25 puntos arriba de ventaja”.
Adelantó que regresará como presidente electo para traer el plan de desarrollo e informará cuánto invertirá en actividades productivas, infraestructura y desarrollo social por pueblo, localidad, municipios y la región, se le dará seguimiento, se evaluará entre todos.
Por la tarde, López Obrador estuvo en Tonalá, Jalisco y mañana visitará Tlaquepaque y Puerto, Jalisco.
"""
dt = nlp(re.sub(r'[\u201c\u201d]', r'"', test))

Wall time: 10.8 s


In [12]:
for i_sent, sent in enumerate(dt.sentences):
    print(f'-------- Oración {i_sent+1} -----------')
    for token in sent.tokens:
        print(f'{token.text}', end='  ')
    print('\n------------------------------------\n')

-------- Oración 1 -----------
Andrés  Manuel  López  Obrador  observó  que  en  el  segundo  debate  le  fue  peor  a  Anaya  que  a  Meade  ,  porque  Anaya  quedó  al  descubierto  como  un  mentecato  ,  un  mentiroso  y  "  nosotros  para  arriba  como  la  espuma  del  mar  "  .  
------------------------------------

-------- Oración 2 -----------
En  entrevista  y  a  la  pregunta  de  los  reporteros  que  ayer  se  apagó  el  amor  y  paz  ,  porque  respondió  a  los  señalamientos  de  Anaya  y  Meade  ,  el  candidato  por  la  coalición  "  Juntos  haremos  historia  "  manifestó  que  se  pasó  de  la  raya  el  candidato  del  PAN  .  
------------------------------------

-------- Oración 3 -----------
"  Me  dijeron  de  que  era  amor  y  paz  ,  pero  que  yo  no  de  dejará  ,  mis  asesores  del  pueblo  me  aconsejaron  ,  me  dijeron  sí  se  portan  altaneros  y  groseros  ,  ponlos  en  su  lugar  ,  dales  un  estate  quieto  "  ,  dijo  .  
-----------------

In [5]:
client.close()

In [ ]:

for i_sent, sent in enumerate(doc.sentences):
    print(f'-------- Oración {i_sent+1} -----------')
    for token in sent.tokens:
#         print(f'token: {token.text}\t\twords: {", ".join([word.text for word in token.words])}')
        print(f'{token.to_dict()}')
    print('------------------------------------\n')